<a href="https://colab.research.google.com/github/nicoRomeroCuruchet/DynamicProgramming/blob/main/testing_bary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
from PolicyIteration import PolicyIteration
from utils.utils import plot_2D_value_function,\
                        plot_3D_value_function,\
                        test_enviroment

# CartPoleEnv 

### Observation Space

The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

| Num | Observation           | Min                 | Max               |
|-----|-----------------------|---------------------|-------------------|
| 0   | Cart Position         | -4.8                | 4.8               |
| 1   | Cart Velocity         | -Inf                | Inf               |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3   | Pole Angular Velocity | -Inf                | Inf               |

### Action Space

The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction
of the fixed force the cart is pushed with.

- 0: Push cart to the left
- 1: Push cart to the right

In [2]:
# Train cartpole environment:
from classic_control.cartpole import CartPoleEnv

env = CartPoleEnv(sutton_barto_reward=True)
# position thresholds:
x_lim         = 2.4
theta_lim     = 0.418 
# velocity thresholds:
x_dot_lim     = 3.1
theta_dot_lim = 3.1

bins_space = {
    "x_space"         : np.linspace(-x_lim, x_lim, 10,  dtype=np.float32),                     # position space          (0)
    "x_dot_space"     : np.linspace(-x_dot_lim, x_dot_lim, 10,  dtype=np.float32),             # velocity space          (1)
    "theta_space"     : np.linspace(-theta_lim, theta_lim, 10, dtype=np.float32),              # angle space             (2)
    "theta_dot_space" : np.linspace(-theta_dot_lim, theta_dot_lim, 10, dtype=np.float32),      # angular velocity space  (3)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=np.array([0, 1], dtype=np.int32),
    gamma=0.99,
    theta=1e-3
)

#pi.run()

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
2024-09-17 16:07:48.995 | INFO     | PolicyIteration:__init__:129 - Lower bounds: [-2.4   -3.1   -0.418 -3.1  ]
2024-09-17 16:07:48.998 | INFO     | PolicyIteration:__init__:130 - Upper bounds: [2.4   3.1   0.418 3.1  ]
2024-09-17 16:07:49.188 | INFO     | PolicyIteration:__init__:142 - The action space is: [0 1]
2024-09-17 16:07:49.189 | INFO     | PolicyIteration:__init__:143 - Number of states: 10000
2024-09-17 16:07:49.190 | INFO     | PolicyIteration:__init__:144 - Total states:20000
2024-09-17 16:07:49.191 | INFO     | PolicyIteration:__init__:147 - Creating Delaunay triangulation...
2024-09-17 16:07:58.105 | INFO     | PolicyIteration:__init__:149 - Delaunay triangulation created.
2024-09-17 16:07:58.176 | INFO     | PolicyIteration:__init__:171 - Policy Iteration was correctly initialized.
2024-09-17 16:07:58.177 | INFO     | PolicyIteration:__init__:172 - The envirome

In [ ]:
# Test cartpole environment:

with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi = pickle.load(f)

test_enviroment(CartPoleEnv(sutton_barto_reward=True, render_mode="human"), pi)

## Observation Space

The observation is a `ndarray` with shape `(2,)` where the elements correspond to the following:

| Num | Observation                          | Min   | Max  | Unit         |
|-----|--------------------------------------|-------|------|--------------|
| 0   | position of the car along the x-axis | -1.2  | 0.6  | position (m) |
| 1   | velocity of the car                  | -0.07 | 0.07 | velocity (v) |

## Action Space

There are 3 discrete deterministic actions:

- 0: Accelerate to the left
- 1: Don't accelerate
- 2: Accelerate to the right


In [ ]:
from classic_control.continuous_mountain_car import Continuous_MountainCarEnv

env=Continuous_MountainCarEnv()

bins_space = {
    "x_space":     np.linspace(env.min_position, env.max_position, 100,      dtype=np.float32),    # position space    (0)
    "x_dot_space": np.linspace(-abs(env.max_speed), abs(env.max_speed), 100, dtype=np.float32),    # velocity space    (1)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=np.linspace(-1.0, +1.0,9, dtype=np.float32),
    gamma=0.99,
    theta=1e-3,
)
pi.run() # 250*

In [ ]:
# Test mountain car environment:
with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi: PolicyIteration = pickle.load(f)

test_enviroment(Continuous_MountainCarEnv(render_mode="human"), pi)

In [ ]:
# graph the value function of the mountain car environment:
plot_3D_value_function(pi.value_function)
plot_2D_value_function(pi.value_function)

In [3]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()
pi.calculate_transition_reward_table()
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

2024-09-17 16:08:03.747 | WARNING  | PolicyIteration:calculate_transition_reward_table:346 - Some states are outside the bounds of the environment.
2024-09-17 16:08:38.152 | WARNING  | PolicyIteration:calculate_transition_reward_table:346 - Some states are outside the bounds of the environment.


         310862 function calls (307001 primitive calls) in 62.293 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   62.300   31.150 /home/nromero/anaconda3/envs/DynamicProgramming/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3517(run_code)
        2    0.000    0.000   62.300   31.150 {built-in method builtins.exec}
        1    0.000    0.000   62.300   62.300 /tmp/ipykernel_29129/2983300005.py:1(<module>)
        1    0.002    0.002   62.300   62.300 /home/nromero/Documents/DynamicProgramming/src/PolicyIteration.py:325(calculate_transition_reward_table)
        2   61.141   30.570   61.803   30.902 /home/nromero/Documents/DynamicProgramming/src/PolicyIteration.py:185(barycentric_coordinates)
   100/46    0.000    0.000    1.134    0.025 /home/nromero/anaconda3/envs/DynamicProgramming/lib/python3.11/site-packages/jax/_src/traceback_util.py:176(reraise_with_filtered_traceback)


In [ ]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()
pi.run()
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())